In [1]:
# general libraries
import json
import urllib.request
from copy import copy
# biopython libraries
from Bio.Alphabet import generic_dna, generic_protein
## for sequence handling
from Bio import SeqIO
from Bio.Seq import Seq
## for alignments
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo
## for BLAST
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
## for UniProt
from Bio import ExPASy
from Bio import SwissProt

import pandas as pd

from IPython.display import Image

# DNA to protein sequences

## You will KNOW:
* What is a gene in biology
* How does a gene become a protein in the cell
* How a gene is represented in Ensembl
* What is a REST web-service
* Some magic in Jupyter notebooks


## You will BE ABLE TO:
* Retrieve gene sequences with a REST service
* Align biological sequences with the BioPython library
* Visualize alignments in Jalview
* Analyze a protein sequence launching InterProScan remotely
* Find similar sequences launching pHMMER remotely

## We will use
* Python3 (Jupyter in this case but vanilla python2.7/3.x is OK)
* BioPython library
* Jalview [http://www.jalview.org/](http://www.jalview.org/)

### Install python and required libraries (Unix-like) (may require sudo)
`apt install python3`<br>
`apt install python-pip`<br>
`pip install biopython`<br>
`pip install pandas` (just for visualization purposes) <br>
`pip install xmltramp2` (required by InterProScan API script) <br>

### If you want to use Jupyter 
`pip install jupyter`<br>
`jupyter notebook`

### Install Jalview
#### Windows & JRE
* Click on `Launch Jalview Desktop` button on the homepage __[http://www.jalview.org/](http://www.jalview.org/)__
* Execute file

#### Any OS w/o JRE
* Go to __[http://www.jalview.org/Web_Installers/install.htm](http://www.jalview.org/Web_Installers/install.htm)__
* Choose right download from the __includes Java VM__ column: 
* Launch installer



## What is a gene

A gene is a *discrete* *inheritable* unit that gives rise to observable *physical characteristics*.

* **It gives rise to physical characteristics** means that it produces an observable phenomenon (a.k.a. phenotype)
* **Inheritable** means that can be transferred across generations
* **Discrete** means that phenotypes can be inherited singularly

## Discovery of DNA and invention of genetics

1860: The concept of discrete inheritable units was first suggested by Gregor Mendel

1905: The name *gene* comes up from William Bateson

1923: Frederick Griffith observed that DNA carries genes

1941: Edward Lawrie Tatum and George Wells Beadle show that genes code for proteins. This is known as the original **central dogma of molecular biology**

## The central dogma of molecular biology
<img width="80%" src="media/centraldogma.jpg">

A gene is **transcribed** to RNA first, then **translated** to Protein
* DNA sits in the nucleus of the cell and is made of Adenine (A), Thymine (T), Cytosine (C) and Guanine (G) **nucleotides**
* RNA is made in the nucleus and then brought outside. It uses Uracile (U) instead of Thymine (T)
* Proteins are made of **amino acids**, completely different molecules from nucleotides
* A **codon** (3 nucleotides) code for 1 amino acid following the **genetic code**, which is universal across all life forms.

## The structure of DNA

<img align='centetr' width=80% src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/DNA_chemical_structure_2.svg/1200px-DNA_chemical_structure_2.svg.png">


In 1953 DNA structure is resolved to be a double helix by Rosalind Franklin, James Watson and Francis Crick

It's a molecule composed of two chains (made of *nucleotides*) that coil around each other to form a double helix. 

Each chain is a sequence of nucleotides and the two helices are kept together by interactions between complementary nucleotides

## Genes are highly complex units
<img align='center' width=80% src="https://upload.wikimedia.org/wikipedia/commons/5/54/Gene_structure_eukaryote_2_annotated.svg">

There are many portion of the gene that have **regulatory functions**

The portion of a gene that is transcribed to RNA has its own regulatory sequences (UTRs, introns) and need to be processed and trimmed (splicing) beforme becoming a **mature** messenger

Introns are removed and exons are combined to form a mature messenger RNA, which is then **translated** to protein

In [2]:
# from IPython.display import HTML
# HTML('<iframe width="100%" src="https://www.youtube.com/embed/gG7uCskUOrA?ecver=1" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')
# HTML('<iframe width="100%" src="https://www.youtube.com/embed/2BwWavExcFI?ecver=1" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

In [3]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src="media/translation.mp4" type="video/mp4">
</video></div>

## Some numbers
A ribosome translates approximately $20$ amino acids per second (reading $60$ nuceotides per second)

In a typical cell there are around $50\,000$ ribosomes

In a typical human there are around $37$ trillions $(10^{12})$ cells

$$ 20 * (5*10^5) * (37 *10^{12}) = 3.7*10^{20}$$

Just accounting for translation events. Other operations going on at cellular level:
* DNA replication
* Chromosome breathing (folding/unfolding)
* DNA repairing
* Epygenetic modifications (DNA methylation)
* RNA-mediated cell regulation
* Protein post translational modifications
* Enzyme-mediated catalysis or chemical reactions
* ...

## Genes in Ensembl
Genes are stored as sequences representing nucleotides (ACTG)

The central resource containing gene sequences is __[Ensembl](https://www.ensembl.org/index.html)__

Ensembl is an example of **genome browser**, a tool to browse the genome.

It's highly organized and has an elaborate interface to retrieve and __[visualize data](https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000139618;r=13:32315474-32400266)__

Ensembl - like many EBI services - offer a **REST API**

### What is a REST service
Representational State Transfer (**REST**) is a set of constraints to be used for creating web services.

RESTful web services, provide **interoperability** between computer systems on the Internet.

In a RESTful web service, requests are made to a resource's URL (Uniform Resource Locator) or colloquially a **web address**

Requests usually follow the **HTTP** protocol (the operations available are GET, POST, ...)

RESTful web services expose one or more **endpoints** to access different data. 

## Retrieve a gene sequence from Ensembl
* Use the RESTful API to retrieve a gene sequence from the Ensemble database.
* The gene we want to obtain is the *cyclin dependent kinase 20* (`ENSG00000156345`)
* Documentation for Ensembl endpoints is at __[https://rest.ensembl.org/](https://rest.ensembl.org/)__
* There are different kind of sequence we can ask for
    * **genomic**: the DNA sequence
    * **cdna**: complementary DNA (complementary to a messenger RNA)
    * **cds**: coding sequence (complementary to the coding sequence of mRNA)
    * **protein**: protein sequence



In [4]:
base_url = 'https://rest.ensembl.org/'
endpoint = 'sequence/id/'
seq_id = 'ENSG00000156345'
parameters = '?content-type=application/json&type=genomic'
gdna_url = base_url + endpoint + seq_id + parameters

gdna = urllib.request.urlopen(gdna_url).read()
gdna

b'{"molecule":"dna","id":"ENSG00000156345","query":"ENSG00000156345","seq":"GCTGTCATCGTTCCGTGGGCCCTGCTGCGGGCACGCTCTCGGCGCATGCGTTTTTTATGCGGGATTAAGCTTGCTGCTGCGTGACAGCGGAGGGCTAGGAAAAGGCGCAGTGGGGCCCGGAGCTGTCACCCCTGACTCGACGCAGCTTCCGTTCTCCTGGTGACGTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCTAGGCAACAGCGTGCGAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAGTTCAGGGGCACAGGGGCACAGGCCCACGACTGCAGCGGGATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACGGCATCGTCTTCAAGGCCAAGCACGTGGAGGTGAGGCTGGACCGCGGCCGGCAGCCTGGCGGGGGTGTGCCCCCGCCACCCTCCGGCTAACGCTCTAAACTGTTTCGGTTCCCTTTTTACATCCAGTACAGTTTTTAAAACCTACTCATATTCTAAACCTACTTTGGGCCGTTGCGCTTCCCTCCGCACAGCTGGCTTGGTCCCCTACCCCAGCGGCTGGGTCCCAGGCTAGTCCTAGACCCCCGAGGAGGGCCTCTGGCCGAGCCGGGGGCGCGTGTCTCTCTCTCAACCACCTTCCCCTCACCCACCTTCATCTCTCTTTCCCAGCCGAGGGTGGGCTGGCAGTGTCTGCCTTCTATCCTGCAGACTGGCGAGATAGTTGCCCTCAAGAAGGTGGCCCTAAGGCGGTTGGAGGACGGCTTCCCTAACCAGGCCCTGCGGGAGATTAAGGCTCTGCAGGAGATGGAGGACAATCAGTATGTGAGTAGGGGAGGGGGGGCATGGTATTCTCACCCTCAGTCGCTCGTTCCCACTTCTTTGTGCCTTCATTTTCCCAGCTACGCCTTCACCAG

In [5]:
# import json
gdna = json.loads(gdna)
gdna

{'molecule': 'dna',
 'id': 'ENSG00000156345',
 'query': 'ENSG00000156345',
 'seq': 'GCTGTCATCGTTCCGTGGGCCCTGCTGCGGGCACGCTCTCGGCGCATGCGTTTTTTATGCGGGATTAAGCTTGCTGCTGCGTGACAGCGGAGGGCTAGGAAAAGGCGCAGTGGGGCCCGGAGCTGTCACCCCTGACTCGACGCAGCTTCCGTTCTCCTGGTGACGTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCTAGGCAACAGCGTGCGAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAGTTCAGGGGCACAGGGGCACAGGCCCACGACTGCAGCGGGATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACGGCATCGTCTTCAAGGCCAAGCACGTGGAGGTGAGGCTGGACCGCGGCCGGCAGCCTGGCGGGGGTGTGCCCCCGCCACCCTCCGGCTAACGCTCTAAACTGTTTCGGTTCCCTTTTTACATCCAGTACAGTTTTTAAAACCTACTCATATTCTAAACCTACTTTGGGCCGTTGCGCTTCCCTCCGCACAGCTGGCTTGGTCCCCTACCCCAGCGGCTGGGTCCCAGGCTAGTCCTAGACCCCCGAGGAGGGCCTCTGGCCGAGCCGGGGGCGCGTGTCTCTCTCTCAACCACCTTCCCCTCACCCACCTTCATCTCTCTTTCCCAGCCGAGGGTGGGCTGGCAGTGTCTGCCTTCTATCCTGCAGACTGGCGAGATAGTTGCCCTCAAGAAGGTGGCCCTAAGGCGGTTGGAGGACGGCTTCCCTAACCAGGCCCTGCGGGAGATTAAGGCTCTGCAGGAGATGGAGGACAATCAGTATGTGAGTAGGGGAGGGGGGGCATGGTATTCTCACCCTCAGTCGCTCGTTCCCACTTCTTTGTGCCTTCATTTTCCCAGCTACGCC

In [6]:
# items() reutrn an iterator with keys and values
# the format method allow alignments {:>x}
# ternary operators
for k, v in gdna.items():
    print('{:>8}: {}{}'.format(k, str(v)[:40], '... {} more'.format(len(gdna['seq']) - 40) if k == 'seq' else ''))

molecule: dna
      id: ENSG00000156345
   query: ENSG00000156345
     seq: GCTGTCATCGTTCCGTGGGCCCTGCTGCGGGCACGCTCTC... 8273 more
    desc: chromosome:GRCh38:9:87966441:87974753:-1
 version: 17


## Retrieve all transcripts relative to a gene
By using the same endpoint with different parameters we have access to the transcript(s) of a gene. Remember that a single gene may have **multiple** transcripts.

* Sequence endpoint documentation: __[https://rest.ensembl.org/documentation/info/sequence_id](https://rest.ensembl.org/documentation/info/sequence_id)__
* Ensembl REST base url `https://rest.ensembl.org/`
* Sequence endpoint     `sequence/id/`
* Gene id               `ENSG00000156345`
* python function       `urllib.request.urlopen(URL).read()`
* Hint 1: Remember to set the `type` parameter to `cdna` and `cds`
* Hint 2: You need to account for the multiple sequences in the response setting the parameter `multiple_sequences=1`

Try to answer these questions:
* How many cdna transcripts do you find? How many cds transcripts? 
* Can you imagine why the numbers are different? 
* Do you notice any differences in the sequences of cdna and cds transcripts?
* Why do you think they are different?

In [7]:
# import urllib.request, json
# import pandas as pd
params = '?content-type=application/json&type=cdna&multiple_sequences=1'
cdna_url = base_url + endpoint + seq_id + params
cdna = urllib.request.urlopen(cdna_url).read()
pd.DataFrame.from_dict(json.loads(cdna)).set_index('id')

,desc,molecule,query,seq,version
id,,,,,
ENST00000459720,None,dna,ENSG00000156345,GCTTGCTGCTGCGTGACAGCGGAGGGCTAGGAAAAGGCGCAGTGGG...,5
ENST00000603475,None,dna,ENSG00000156345,CGTGCGAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGG...,1
ENST00000336654,None,dna,ENSG00000156345,GTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCT...,9
ENST00000375883,None,dna,ENSG00000156345,GCTGTCATCGTTCCGTGGGCCCTGCTGCGGGCACGCTCTCGGCGCA...,7
ENST00000375871,None,dna,ENSG00000156345,AGTTCAGGGGCACAGGGGCACAGGCCCACGACTGCAGCGGGATGGA...,8
ENST00000605159,None,dna,ENSG00000156345,GGAGAAGTGGAGTTTGGAAGTTCAGGGGCACAGGGGCACAGGCCCA...,5
ENST00000325303,None,dna,ENSG00000156345,CTGTCATCGTTCCGTGGGCCCTGCTGCGGGCACGCTCTCGGCGCAT...,8
ENST00000486228,None,dna,ENSG00000156345,TTTTATGCGGGATTAAGCTTGCTGCTGCGTGACAGCGGAGGGCTAG...,7
ENST00000604175,None,dna,ENSG00000156345,GCGAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAG...,5


In [8]:
cds_url = base_url + endpoint + seq_id + params.replace('cdna', 'cds')
cds = json.loads(urllib.request.urlopen(cds_url).read())
pd.DataFrame.from_dict(cds).set_index('id')

,desc,molecule,query,seq,version
id,,,,,
ENST00000336654,None,dna,ENSG00000156345,ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACG...,9
ENST00000375883,None,dna,ENSG00000156345,ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACG...,7
ENST00000375871,None,dna,ENSG00000156345,ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACG...,8
ENST00000605159,None,dna,ENSG00000156345,ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACG...,5
ENST00000325303,None,dna,ENSG00000156345,ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACG...,8


## Retrieve exons of a gene
Ensembl APIs allow to get the list of exons found in a genomic region throught the `overlap` endpoint. 

This endpoint allows to find sequence features overlapping a genomic regions, in this case `feature=exon`.



In [9]:
exons_url = base_url + 'overlap/id/' + seq_id + '?content-type=application/json&feature=exon'
exons = urllib.request.urlopen(exons_url).read()
exons = json.loads(exons)
pd.DataFrame.from_dict(exons).set_index('id').head()

,Parent,assembly_name,constitutive,end,ensembl_end_phase,ensembl_phase,exon_id,feature_type,rank,seq_region_name,source,start,strand,version
id,,,,,,,,,,,,,,
ENSE00001862627,ENST00000459720,GRCh38,0,87974685,-1,-1,ENSE00001862627,exon,1,9,havana,87974372,-1,1
ENSE00003580906,ENST00000459720,GRCh38,0,87974035,-1,-1,ENSE00003580906,exon,2,9,havana,87973922,-1,1
ENSE00003551454,ENST00000459720,GRCh38,0,87971335,-1,-1,ENSE00003551454,exon,3,9,havana,87971147,-1,1
ENSE00001872570,ENST00000459720,GRCh38,0,87970897,-1,-1,ENSE00001872570,exon,4,9,havana,87969194,-1,1
ENSE00001822512,ENST00000459720,GRCh38,0,87967659,-1,-1,ENSE00001822512,exon,5,9,havana,87966441,-1,1


In [10]:
len(exons)

55

There are so many exons because in Ensembl exons are redundant. Each transcript has its own set of exons even if they are the same exact genomic region.

Exons have parents transcript identified by their `Parent` key.

We can filter them by `Parent` key to select only the exons whose parent is the id of one (or all) transcript(s).

In [11]:
transcript = cds[0]['id']
exons_example = list(filter(lambda d: d['Parent']==transcript, exons))
pd.DataFrame.from_dict(exons_example).set_index('id')

,Parent,assembly_name,constitutive,end,ensembl_end_phase,ensembl_phase,exon_id,feature_type,rank,seq_region_name,source,start,strand,version
id,,,,,,,,,,,,,,
ENSE00001890030,ENST00000336654,GRCh38,0,87974589,0,-1,ENSE00001890030,exon,1,9,ensembl_havana,87974372,-1,1
ENSE00001342208,ENST00000336654,GRCh38,0,87974074,0,0,ENSE00001342208,exon,2,9,ensembl_havana,87973922,-1,2
ENSE00003501098,ENST00000336654,GRCh38,0,87971335,0,0,ENSE00003501098,exon,3,9,ensembl_havana,87971147,-1,1
ENSE00003471288,ENST00000336654,GRCh38,0,87970897,2,0,ENSE00003471288,exon,4,9,ensembl_havana,87970776,-1,1
ENSE00003508198,ENST00000336654,GRCh38,0,87969919,0,2,ENSE00003508198,exon,5,9,ensembl_havana,87969796,-1,1
ENSE00003535476,ENST00000336654,GRCh38,0,87969349,0,0,ENSE00003535476,exon,6,9,ensembl_havana,87969194,-1,1
ENSE00001902992,ENST00000336654,GRCh38,0,87967659,-1,0,ENSE00001902992,exon,7,9,ensembl_havana,87966446,-1,1


### Exons usage
Not all transcript use the same exons. 

Some transcripts are shorter because they are assmebled starting from different combination of exons.

Furthermore, exons can be assembled in different orders.

Luckily not all combinations are viable

In [12]:
exons_per_transcript = [[tr['id']] + [e['id'] for e in filter(lambda d: d['Parent'] == tr['id'], exons)] for tr in cds]
pd.DataFrame(exons_per_transcript).set_index(0).transpose()

,ENST00000336654,ENST00000375883,ENST00000375871,ENST00000605159,ENST00000325303
1,ENSE00001890030,ENSE00003592303,ENSE00003553919,ENSE00003641963,ENSE00001468688
2,ENSE00001342208,ENSE00003694606,ENSE00003694606,ENSE00003694606,ENSE00003694606
3,ENSE00003501098,ENSE00003501098,ENSE00003501098,ENSE00003501098,ENSE00003501098
4,ENSE00003471288,ENSE00003471288,ENSE00003471288,ENSE00003471288,ENSE00003471288
5,ENSE00003508198,ENSE00003508198,ENSE00003658341,ENSE00003508198,ENSE00003532544
6,ENSE00003535476,ENSE00003535476,ENSE00001701771,ENSE00003535476,ENSE00003508198
7,ENSE00001902992,ENSE00001902992,None,ENSE00003527648,ENSE00003535476
8,None,None,None,None,ENSE00001902501


## Visualization of exon usage in Ensembl
This is an example of the information found on a genome browser. In this case, exon usage for different transcripts is shown in a intuitive way. 

<div style='overflow-y: scroll; max-height: 400px'><img align='left' width=100% src="media/Human_98796627587974919.png"></div>

## Biopython
### Writing exons in a FASTA file
Biologists love text files! One of them (the most frequently used) is the FASTA format. It's a very simple format to store nucleotide or amino acid sequences. It can contain a single or multiple sequence record. Each sequence record is composed of two parts:
* **Header** Identified by its first character `>`
* **Sequence**: any other line

In [13]:
with open('exons.fasta', 'w') as of:
    for exon_id in exons_per_transcript[0]:
        exon_seq_url = base_url + 'sequence/id/' + exon_id + '?content-type=text/x-fasta'
        exon_fasta = urllib.request.urlopen(exon_seq_url).read().decode('utf-8')
        of.write(exon_fasta + '\n')
    print(exon_fasta[:250] + '...')

>ENSE00001902992.1 chromosome:GRCh38:9:87966446:87967659:-1
GCTCTCCTCCATCAGTACTTCTTCACAGCTCCCCTGCCTGCCCATCCATCTGAGCTGCCG
ATTCCTCAGCGTCTAGGGGGACCTGCCCCCAAGGCCCATCCAGGGCCCCCCCACATCCAT
GACTTCCACGTGGACCGGCCTCTTGAGGAGTCGCTGTTGAACCCAGAGCTGATTCGGCCC
TTCATCC...


In order to comply with biologists' love for text files, computer scientists came up with different solutions.

One of them is __[BioPython](https://biopython.org/)__, a python library that let's you manage all kinds of biological file formats.

It also has some useful utilities to:
* align sequences
* launch BLAST remotely
* handle protein structures


In [14]:
# parse FASTA file with biopython module SeqIO
seq_records = list(SeqIO.parse("exons.fasta", "fasta"))
# select one exon from the list of results
an_exon = seq_records[1]
# __repr__ method of Fasta record objects
print(an_exon)

ID: ENSE00001890030.1
Name: ENSE00001890030.1
Description: ENSE00001890030.1 chromosome:GRCh38:9:87974372:87974589:-1
Number of features: 0
Seq('GTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCTAGGCAACA...GAG', SingleLetterAlphabet())


In [15]:
# __repr__ method of the Seq objects
print(an_exon.seq)

GTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCTAGGCAACAGCGTGCGAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAGTTCAGGGGCACAGGGGCACAGGCCCACGACTGCAGCGGGATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACGGCATCGTCTTCAAGGCCAAGCACGTGGAG


In [16]:
# transcription is the conversion of DNA to messenger RNA
print(an_exon.seq.transcribe())

GUCGCCUACAGGAACCGCCCCAGUGGUCAGCUGCCGCGCUGUUGCUAGGCAACAGCGUGCGAGCUCAGAUCAGCGUGGGGUGGAGGAGAAGUGGAGUUUGGAAGUUCAGGGGCACAGGGGCACAGGCCCACGACUGCAGCGGGAUGGACCAGUACUGCAUCCUGGGCCGCAUCGGGGAGGGCGCCCACGGCAUCGUCUUCAAGGCCAAGCACGUGGAG


In [17]:
# translation is the conversion of messenger RNA to amino acid
print(an_exon.seq.transcribe().translate())

VAYRNRPSGQLPRCC*ATACELRSAWGGGEVEFGSSGAQGHRPTTAAGWTSTASWAASGRAPTASSSRPSTW


/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2576: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [18]:
def print_multiline_alignment(alignment, n=60):
    seq1 = [alignment[0][i:i+n] for i in range(0, len(alignment[0]), n)]
    seq2 =  [alignment[1][i:i+n] for i in range(0, len(alignment[1]), n)]
    nstart1, nend1 = 0, 0
    nstart2, nend2 = 0, 0
    for i, (s1, s2) in enumerate(zip(seq1, seq2)):
        gap1 = s1.count('-') 
        gap2 = s2.count('-')
        if i == 0:
            nstart1 = 1 if gap1 != n else nstart1
            nstart2 = 1 if gap2 != n else nstart2
            nend1 = n - gap1 if gap1 != n else nend1
            nend2 = n - gap2 if gap2 != n else nend2

        print('{:>5} {:>5} {} {}'.format('tmplt', nstart1, s1, nend1))
        nstart1 += n - gap1
        nend1 += n - gap1
        nend2 += n - gap2
        print('{:>5} {:>5} {}'.format('', '', ''.join(['|' if x == y else ' ' for x, y in zip(s1, s2)])))
        print('{:>5} {:>5} {} {}'.format('query', nstart2, s2, nend2))
        nstart2 += n - gap2 
        print()

## Pairwise alignment
Biopython has functions to perform pairwise alignments. It has different functions:
* globalxx 
* globalms
* localds...

where global and local indicate the type of alignment and the two letters are match and gap-penalty parameters

The **match** parameters are:
* `x`     No parameters. Identical characters have score of 1, otherwise 0.
* `m`     A match score is the score of identical chars, otherwise mismatch score.
* `d`     A dictionary returns the score of any pair of characters.

The **gap penalty** parameters are:
* `x`     No gap penalties.
* `s`     Same open and extend gap penalties for both sequences.
* `d`     The sequences have different open and extend gap penalties.



In [19]:
# from Bio import pairwise2
alignment = pairwise2.align.globalxx(cds[0]['seq'], an_exon.seq)[0]
# custom print function
print_multiline_alignment(alignment)

tmplt     1 ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACGGCATCGTCTTCAAG 60
                                                                        
query     0 ------------------------------------------------------------ 0

tmplt    61 GCCAAGCACGTGGAGCCGAGGGTGGGCTGGCAGTGTCTGCCTTCTATCCTGCAGACTGGC 120
                                                                        
query     0 ------------------------------------------------------------ 0

tmplt   121 GAGATAGTTGCCCTCAAGAAGGTGGCCCTAAGGCGGTTGGAGGACGGCTTCCCTAACCAG 180
                                                                        
query     0 ------------------------------------------------------------ 0

tmplt   181 GCCCTGCGGGAGATTAAGGCTCTGCAGGAGATGGAGGACAATCAGTATGTGGTACAACTG 240
                                                |    ||        |     |  
query     0 ------------------------------------G----TC--------G-----C-- 5

tmplt   241 AAGGCTGTGTTCCCACACGGTGGAGGCTTTGTGCTGGCCTTTGAGTTCATGCTGTCGGAT 300
                |   

## Aligning two sequences

By using the the `pairwise2` module, correctly align the **cds** `ENST00000336654` and **exon** `ENSE00001890030.1`

Retrieve the sequence of the two entities using the endpoint:<br>__[https://rest.ensembl.org/documentation/info/sequence_id](https://rest.ensembl.org/documentation/info/sequence_id)__

Find the correct way to align the two sequences. Documentation of the pairwise2 module is here: <br>__[http://biopython.org/DIST/docs/api/Bio.pairwise2-module.html](http://biopython.org/DIST/docs/api/Bio.pairwise2-module.html)__

* Hint 1: function to print alignments __[https://github.com/marnec/practical](https://github.com/marnec/practical)__
* Hint 2: local alignment is the way to go
* Hint 3: You probably want a substitution matrix (http://biopython.org/DIST/docs/api/Bio.SubsMat.MatrixInfo-module.html)
* Hint 4: `print(MatrixInfo.available_matrices)`

In [20]:
# x = Identical characters have score of 1, otherwise 0
# s = Same open and extend gap penalties for both sequences
alignment = pairwise2.align.localxs(cds[0]['seq'], an_exon.seq, -1, -0.5)[0]
print_multiline_alignment(alignment)

tmplt     1 ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGGGCGCCCACGGCATCGTCTTCAAG 60
                                                                        
query     0 ------------------------------------------------------------ 0

tmplt    61 GCCAAGCACGTGGAGCCGAGGGTGGGCTGGCAGTGTCTGCCTTCTATCCTGCAGACTGGC 120
                                                                        
query     0 ------------------------------------------------------------ 0

tmplt   121 GAGATAGTTGCCCTCAAGAAGGTGGCCCTAAGGCGGTTGGAGGACG-GCT-TCCCTAACC 180
                  || |||  || |||    |||| |  | |||  |  | || ||| |  |||  |
query     0 ------GTCGCCTACAGGAA--CCGCCCCA--GTGGTCAGCTGCCGCGCTGTTGCTAGGC 50

tmplt   179 AGGCCCTGCGGGAGATTAAGGCTCTGCAGGAGATGGAGGACAATCAGTATGTGGTACAAC 238
            |  |   | | ||| | |  | || ||  | | ||||||| ||   |  | ||| |   |
query    50 A-ACAGCGTGCGAGCTCA--GATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAGTTC 107

tmplt   239 TGAAGGCTGTGTTCC-CACACGGTGGAG-GCTTTGTGCTGGCCTTTGAGTTCATGCT--G 298
             |  |

In [21]:
# from Bio.SubsMat import MatrixInfo
## d = A dictionary returns the score of any pair of characters
## s = Same open and extend gap penalties for both sequences
alignment = pairwise2.align.localds(cds[0]['seq'], an_exon.seq, MatrixInfo.genetic, -1000, -500)[0]
print_multiline_alignment(alignment)

tmplt     0 ------------------------------------------------------------ 0
                                                                        
query     1 GTCGCCTACAGGAACCGCCCCAGTGGTCAGCTGCCGCGCTGTTGCTAGGCAACAGCGTGC 120

tmplt     0 ------------------------------------------------------------ 0
                                                                        
query    61 GAGCTCAGATCAGCGTGGGGTGGAGGAGAAGTGGAGTTTGGAAGTTCAGGGGCACAGGGG 180

tmplt     0 -----------------------ATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGG 0
                                   |||||||||||||||||||||||||||||||||||||
query   121 CACAGGCCCACGACTGCAGCGGGATGGACCAGTACTGCATCCTGGGCCGCATCGGGGAGG 240

tmplt    37 GCGCCCACGGCATCGTCTTCAAGGCCAAGCACGTGGAGCCGAGGGTGGGCTGGCAGTGTC 37
            ||||||||||||||||||||||||||||||||||||||                      
query   181 GCGCCCACGGCATCGTCTTCAAGGCCAAGCACGTGGAG---------------------- 278

tmplt    97 TGCCTTCTATCCTGCAGACTGGCGAGATAGTTGCCCTCAAGAAGGTGGCCCTAAGGCGGT 97
                   

### Take home message: Instruments & meaning

Possibilities might seem endless, however not all paths of action make sense.

Knoweldge of the **origin** and **meaning** of data is often **essential**.

When align two sequences keep in mind what they are and the relationship between them.

## Multiple sequence alignment

Aligning $n$ sequences with dynamic programming implies performing $\frac{n(n-1)}{2}$ pairwise alignments.

This is an $O(n^2)$ problem. Other methods are used, among which Clustal Omega is very fast and accurate.

BioPython doesn't offer a way to perform multiple sequence alignments.

To make it quick let's go on Clustal Omega website __[https://www.ebi.ac.uk/Tools/msa/clustalo/](https://www.ebi.ac.uk/Tools/msa/clustalo/)__

We will then visualize alignments in Jalview __[http://www.jalview.org/](http://www.jalview.org/)__

In [22]:
base_url = 'https://rest.ensembl.org/'
endpoint = 'sequence/id/'
seq_id = 'ENSG00000156345'
# set the parameters to retrieve protein sequence in FASTA format
params = '?content-type=text/x-fasta&type=protein&multiple_sequences=1'
protein_url = base_url + endpoint + seq_id + params
protein = urllib.request.urlopen(protein_url).read().decode('utf-8')
# wirte response directly to file

In [23]:
print(protein[:500])

>ENSP00000338975.5
MDQYCILGRIGEGAHGIVFKAKHVEPRVGWQCLPSILQTGEIVALKKVALRRLEDGFPNQ
ALREIKALQEMEDNQYVVQLKAVFPHGGGFVLAFEFMLSDLAEVVRHAQRPLAQAQVKSY
LQMLLKGVAFCHANNIVHRDLKPANLLISASGQLKIADFGLARVFSPDGSRLYTHQVATR
SVGCIMGELLNGSPLFPGKNDIEQLCYVLRILGTPNPQVWPELTELPDYNKISFKEQVPM
PLEEVLPDVSPQALDLLGQFLLYPPHQRIAASKALLHQYFFTAPLPAHPSELPIPQRLGG
PAPKAHPGPPHIHDFHVDRPLEESLLNPELIRPFILEG
>ENSP00000365043.3
MDQYCILGRIGEGAHGIVFKAKHVETGEIVALKKVALRRLEDGFPNQALREIKALQEMED
NQYVVQLKAVFPHGGGFVLAFEFMLSDLAEVVRHAQRPLAQAQVKSYLQMLLKGVAF


In [24]:
with open('proteins.fasta', 'w') as of:
    of.write(protein + '\n')

## Aligning protein sequences against protein databases
Searching a sequence in a protein database means finding the best alingment of a query sequence and up to million of targets.

HMMER is a project offering many tools to perform searches agains HMM profiles<br>__[https://www.ebi.ac.uk/Tools/hmmer/](https://www.ebi.ac.uk/Tools/hmmer/)__

There are different flavors of HMMER softwares:
* **phmmer** aligns protein sequences vs protein database (PDB, Trembl, SwissProt)
* **hmmscan** aligns protein sequences vs HMM database (Pfam, Gene3D, Tigrfam)
* **hmmsearch** aligns a MSA/HMM profile vs protein database (PDB, Trembl, SwissProt)
* **jackhmmer** iteratively searches seq/MSA/HMM vs protein database (PDB, Trembl, SwissProt)

### Exercise: Align unknown sequence to biological DBs

You will all have the same **unkown sequence**. The sequence is in the `unknown.fasta` file in moodle<br>
Can you answer these questions?
* __Are there similar sequences in the SwissProt?__
    * HINT: Use phmmer to find similar sequences in the SwissProt __[https://www.ebi.ac.uk/Tools/hmmer/search/phmmer](https://www.ebi.ac.uk/Tools/hmmer/search/phmmer)__

* __How similar are the first 30 sequences among them?__ 
    * HINT 1: Download fasta, copy 30 sequences in a new file.
    * HINT 2: Perform a multiple sequence alignment from the Clustal Omega website __[https://www.ebi.ac.uk/Tools/msa/clustalo/](https://www.ebi.ac.uk/Tools/msa/clustalo/)__

* __Are there any conserved regions in the sequence?__
    * HINT: Download the alignment, open in Jalview and look at the conservation section

* __Can you obtain additional information from the alignment?__    
    * HINT 1: Build a phylogenetic tree
    * HINT 2: Try to refine the alignment

HMMER can also be downloaded as a standalone program or launched from command line 

Documentation for hmmer at __[https://hmmer-web-docs.readthedocs.io/en/latest/api.html](https://hmmer-web-docs.readthedocs.io/en/latest/api.html)__

In [25]:
# anything appearing after '!' on a line will be executed by the sub-system command-line.
# redirect results of the command assigning the call to a variable
result = !curl -L -H 'Expect:' \
               -H 'Accept:text/xml' -F seqdb=pdb \
               -F algo=phmmer \
               -F seq='<testP.fa' \
               https://www.ebi.ac.uk/Tools/hmmer/search/phmmer
result[:10]

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '100  2316  100    68  100  2248     66   2188  0:00:01  0:00:01 --:--:--  2255',
 '100  2316  100    68  100  2248     66   2186  0:00:01  0:00:01 --:--:--  2255',
 '',
 '  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0',
 '  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0<opt>',
 '  <data name="results" algo="phmmer" uuid="2CB702F8-E1AF-11E8-8FD7-2CA053F04F9B">']

In [26]:
# ugly solution (but it works)
for line in result:
    if 'uuid' in line:
        uuid = line.split('=')[-1][1:-2]
uuid

'2CB702F8-E1AF-11E8-8FD7-2CA053F04F9B'

In [27]:
%%bash -s "$uuid" --out hmmer_result

# bash environment I can't write this command to the top
# -s pass positional arguments from notebook namespace
# --out redirect output to variable

curl -L -H 'Expect:' \
     -H 'Accept:text/xml' \
     https://www.ebi.ac.uk/Tools/hmmer/results/"$1"?output=json&ali=1&range=1,1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    65  100    65    0     0    201      0 --:--:-- --:--:-- --:--:--   201
100  387k  100  387k    0     0   210k      0  0:00:01  0:00:01 --:--:--  550k


In [28]:
# very complex output, I just want a summary
header = ['acc', 'ndom', 'desc', 'pvalue']
tab = []
for hit in json.loads(hmmer_result)['results']['hits']:
    tab.append([hit[field] for field in header])
    
result_table = pd.DataFrame(tab)
result_table.columns = header
result_table.set_index('acc').head(10)

,ndom,desc,pvalue
acc,,,
1i3q_A,1,DNA-DIRECTED RNA POLYMERASE II LARGEST SUBUNIT,-2776.457219
3rzd_A,1,DNA-directed RNA polymerase II subunit RPB1,-2776.457219
5ip7_A,1,DNA-directed RNA polymerase II subunit RPB1,-2774.861284
4a3b_A,1,DNA-DIRECTED RNA POLYMERASE II SUBUNIT RPB1,-2774.860097
3j1n_A,1,DNA-directed RNA polymerase II subunit RPB1,-2323.344860
3j0k_A,1,DNA-directed RNA polymerase II largest subunit,-2323.344860
5x4z_A,1,DNA-directed RNA polymerase subunit,-2002.511940
3h0g_A,1,DNA-directed RNA polymerase II subunit rpb1,-1737.379536
5u0s_a,1,RNA polymerase II subunit Rpb1,-1737.379536


## InterProScan
Proteins perform a function in the cell and that function can be inferred from the protein sequence.

InterPro provides functional analysis of proteins by classifying them into **families** and predicting **domains** and important sites.

InterProScan is the online tool that produces InterPro annotations. It aggregates many different tools coming from different research groups.

Endpoints for the RESTful API are not written anywhere. If you search for a REST service you will be redirected here:<br>
https://goo.gl/4XMzPK

Where you'll find directions to a python script<br>
https://github.com/ebi-wp/webservice-clients/blob/master/python/iprscan5.py

In [29]:
# use biopython to obtain a protein sequence from one of our transcripts
protein_seq = Seq(cds[0]['seq']).transcribe().translate()

# pip install xmltramp2
# use jupyter magic to feed a namespace variable to the command line
%run ipr5API.py --outfile interproscanOut --sequence $protein_seq --email myemail@gmail.com --quiet

iprscan5-R20181106-103158-0125-33498173-p2m
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
FINISHED


interproscanOut.out.txt
interproscanOut.log.txt
interproscanOut.tsv.txt
interproscanOut.xml.xml
interproscanOut.htmltarball.html.tar.gz
interproscanOut.gff.txt
interproscanOut.svg.svg
interproscanOut.sequence.txt
interproscanOut.json.txt


In [30]:
df = pd.read_csv('interproscanOut.tsv.txt', delimiter='\t', header=None, usecols=range(2,13), index_col=1)
df.columns = ['length', 'sign acc', 'sign desc', 'start', 'end', 'score', 'status', 'date', 'ipr acc', 'ipr desc']
df.sort_index()

,length,sign acc,sign desc,start,end,score,status,date,ipr acc,ipr desc
1,,,,,,,,,,
CDD,338,cd07832,STKc_CCRK,3,281,1.12966E-173,T,06-11-2018,NaN,NaN
Gene3D,338,G3DSA:3.30.200.20,NaN,1,97,9.4E-21,T,06-11-2018,NaN,NaN
Gene3D,338,G3DSA:1.10.510.10,NaN,182,288,5.6E-23,T,06-11-2018,NaN,NaN
Gene3D,338,G3DSA:1.10.510.10,NaN,99,181,2.8E-24,T,06-11-2018,NaN,NaN
PANTHER,338,PTHR24056:SF242,NaN,169,296,1.7E-73,T,06-11-2018,NaN,NaN
PANTHER,338,PTHR24056:SF242,NaN,4,180,1.7E-73,T,06-11-2018,NaN,NaN
PANTHER,338,PTHR24056,NaN,169,296,1.7E-73,T,06-11-2018,NaN,NaN
PANTHER,338,PTHR24056,NaN,4,180,1.7E-73,T,06-11-2018,NaN,NaN
Pfam,338,PF00069,Protein kinase domain,7,180,9.7E-35,T,06-11-2018,NaN,NaN


More info about the tools used can be found in the about page of InterPro website <br> __[https://www.ebi.ac.uk/interpro/](https://www.ebi.ac.uk/interpro/)__

Let's look for our protein in InterPro. We need a UniProt accession number.

Retrieve UniProt accessions relative to a gene from Ensembl `xrefs/id` endpoint (`ENSG00000156345`)

In [31]:
xref_url = base_url + 'xrefs/id/ENSG00000156345' + '?content-type=application/json&external_db=Uniprot_gn'
xldf = pd.DataFrame.from_dict(json.loads(urllib.request.urlopen(xref_url).read()))
xldf

,db_display_name,dbname,description,display_id,info_text,info_type,primary_id,synonyms,version
0,UniProtKB Gene Name,Uniprot_gn,,CDK20,,DEPENDENT,Q8IZL9,[],0
1,UniProtKB Gene Name,Uniprot_gn,,CDK20,,DEPENDENT,A0A0S2Z5B6,[],0


Every entry in UniProt have precalculated InterPro results in a dedicated page <br> 
__[https://www.ebi.ac.uk/interpro/protein/Q8IZL9](https://www.ebi.ac.uk/interpro/protein/Q8IZL9)__

If you compare InterPro annotation found on the web page to the table we retrieved from the REST service you can notice there are differences in the annotation. This leads us to our third exercise ...

### Why is annotation different?

A short summary up to this point:
1. Retrieved a gene sequence from Ensembl
2. Selected a trasnript among those produced by that gene
3. Translated the transcript in protein sequence with BioPython
4. Scanned that protein sequence for domains with InterProScan
5. Compared InterPro annotation we produced with that precalculated for the protein in UniProt

In [32]:
# from Bio import ExPASy
# from Bio import SwissProt
# xldf = pd.DataFrame.from_dict(json.loads(urllib.request.urlopen(xref_url).read()))
handle = ExPASy.get_sprot_raw(xldf['primary_id'][0])
record = SwissProt.read(handle)

What's the length of the sequence?
* explore attributes of `record` using the `dir()` function: `dir(record)`
* find the correct attribute

In [33]:
# remove built-ins 
attr = list(filter(lambda e: not e.startswith('__'), dir(record)))
# print on three columns
for a, b, c in zip(attr[::3], attr[1::3], attr[2::3]):
    print('{:<30}{:<30}{:<}'.format(a,b,c))

accessions                    annotation_update             comments
created                       cross_references              data_class
description                   entry_name                    features
gene_name                     host_organism                 host_taxonomy_id
keywords                      molecule_type                 organelle
organism                      organism_classification       protein_existence
references                    seqinfo                       sequence
sequence_length               sequence_update               taxonomy_id


In [34]:
record.sequence_length

346

In [35]:
df['length'][0]

338

Lengths are different because the transcript we used for the analysis is not the canonical sequence in UniProt:

* UniProt is a DB of protein sequences
* Each entry is refers to a gene
* However each gene may code for multiple proteins (isoforms)
* Thus each entry has one main isoform - the canonical sequence - and may have additional isoforms.

### Take home message: versioning

Versions of biological entities are very important. They change continuously according to new discoveries and data.

It's very important to know what version of an object you are talking about.